In [409]:
import numpy as np

In [410]:
def weights(i, save = False):
    n = 15
    if i == "input":
        weight = np.random.rand(9,n)
        return weight
    if i == "mid":
        weight = np.random.rand(n,n)
        return weight
    if i == "output":
        weight = np.random.rand(n,9)
        return weight
    if i == "pred_output":
        weight = np.random.rand(n,1)
        return weight

    

In [426]:
# neural (player) = 1 , X's : RNG = 0 or 2, O's
# thinker1 tied to index = 1

class Game():
    def __init__(self , board = "000000000" , player_win_count = 0 , computer_win_count = 0 , tie_count = 0 , end = False, thinker1 = None , 
                 thinker2 = None , predictor = None, over = False , reinforce = 0.5):
        self.board = board
        self.player_win_count = player_win_count
        self.computer_win_count = computer_win_count
        self.tie_count = tie_count
        self.end = end
        self.thinker1 = thinker1
        self.thinker2 = thinker2   
        self.over = over
        self.reinforce1 = reinforce
        self.reinforce2 = reinforce
        self.predictor = predictor

    def reset(self):
        self.player_win_count = 0
        self.computer_win_count = 0
        self.tie_count = 0
        
    def place(self , obj, pos):
        self.board = self.board[:pos] + obj + self.board[pos+1:]
        
    def evaluator(self , index):
        win = index * 3
        diag_ru = self.board[0] + self.board[4] + self.board[8] 
        diag_ld = self.board[2] + self.board[4] + self.board[6]
        if diag_ld == win or diag_ru == win:
            self.winning(index)
            return
        for i in range(3):
            row = self.board[3*i:3*i+3] 
            col = self.board[i] + self.board[i + 3] + self.board[i+6]
            if row == win or col == win:
                self.winning(index)
                return
        
    def winning(self , index):
        if index == "1":
            self.player_win_count += 1
            #print("PLAYER WINS!" , self.player_win_count)
            self.reinforce1 = 1
            self.reinforce2 = 0
        if index == "2":
            self.computer_win_count += 1
            #print("COMPUTER WINS!" , self.computer_win_count)
            self.reinforce1 = 0
            self.reinforce2 = 1
        self.end = True
    
    def play(self , index , show):
        for count in range(1,10):
            if index == 3:
                index = 1
            self.turn(index = index , count = count)
            self.show_board(show)
            if show == True:
                self.predictor(show = True)
            else:
                self.predictor()
            index += 1
            if self.end == True:
                self.end = False
                break
            if count == 9:
                self.tie_count += 1
                #print("THATS A TIE!" , self.tie_count)
                self.reinforce1 = 0.1
                self.reinforce2 = 0.1
                break
            
    def turn(self , index , count):
        pos = self.decision(index)
        index = str(index)
        self.place(index, pos)
        if count  > 4:   # don't bother evaluating for first 4 moves
            self.evaluator(index)
            
    def show_board(self , show = False):
        if self.thinker1 ==  self.inputter or self.thinker2 ==  self.inputter or show == True:
            show = "░" * 9
            for i in range(9):
                if self.board[i] == "1":
                    show = show[:i] + "X" + show[i + 1:]
                if self.board[i] == "2":
                    show = show[:i] + "O" + show[i + 1:]
            show = show[:3]  + "\n" + show[3:6] + "\n" + show[6:9] 
            print(show)
            print(self.board)
            print("--------------------------------------------------")
            print()
    
    def decision(self , index):
        if index == 1:
            pos = self.thinker1()
            return pos
        if index == 2:
            pos = self.thinker2()
            return pos
        
    def rng(self , r = -6):
        if r == -6:
            while True:
                pos = np.random.randint(0,9)
                if self.board[pos] == "0":
                    return pos
            
    def programmed_1(self):
        for i in range(9):
            pos = i
            if self.board[pos] == "0":
                return pos
            
    def programmed_2(self):
        for i in [4,0,8,2,6,1,3,5,7,9]:
            pos = i
            if self.board[pos] == "0":
                return pos
            
    def inputter(self , r = -6):
        if r == -6:
            pos = int(input("pick a number 1 - 9: ")) - 1
            return pos
            
    def play_again(self , j , thinker1 , thinker2, predictor, index = 1 , automate = False , show = False):  # index can be one or two
        self.thinker1 = thinker1
        self.thinker2 = thinker2
        self.predictor = predictor
        for i in range(j):
            if i > 0:
                self.over = True
                self.thinker1(self.reinforce1)
                self.thinker2(self.reinforce2)
                self.predictor(self.reinforce1)
                self.over = False
            if automate == False:
                another = input("play again? [y/n]: ")
            else:
                another = "y"
            if another == "y":
                self.board = "0" * 9
                self.play(index , show)
                if show == True:
                    print("------------------")
            else:
                break
        print(self.player_win_count , self.computer_win_count , self.tie_count )
        self.reset()
            
    

In [427]:
# must recieve data about board, and back_propogation
# must send data about position chosen


class Neural(Game):
    def __init__ (self , G = Game , save_data = [] , weight_input = weights("input") , weight_mid = weights("mid") , weight_output = weights("output")):
        self.save_data = save_data
        self.weight_input = weight_input
        self.weight_mid = weight_mid
        self.weight_output = weight_output
        self.G = G
    
    def neural_1(self , reinforce = 0):
        if self.G.over == True:
            self.back_propogate(reinforce)
            self.save_neural_choices_reset()
        else:
            bias = 0.1
            input_data = self.neural_row1(self.G.board)
            layer2_data = self.sigmoid(input_data.dot(self.weight_input))  # 1 X 9 times 9 X 12 --> 1 X 12
            layer3_data = self.sigmoid(layer2_data.dot(self.weight_mid))  # 1 X 12 times 12 X 12 --> 1 X 12
            output_data = self.sigmoid(layer3_data.dot(self.weight_output))  # 1 X 12 times 12 X 9 --> 1 X 9
            choices = list(enumerate(output_data[0]))
            #print(choices)
            #print("------------")
            while True:
                choice = max(choices , key = lambda x: x[1])
                pos = choice[0]
                if self.G.board[pos] == "0":
                    self.save_neural_choices(input_data, layer2_data , layer3_data , output_data , pos)
                    return pos
                else:
                    choices.remove(choice)
        
    def neural_row1(self , board):    
        input_data = np.array([list(map(float , list(board)))])   # makes the board as a string into number
        input_data = input_data.astype(float)
        np.place(input_data , input_data==2 , -1)   # opponent is viewed as low value
        np.place(input_data , input_data==0 , 0)   # empty space is regular value
        np.place(input_data , input_data==1 , 1)
        return input_data
    
    def save_neural_choices(self , input_data , layer2_data , layer3_data , output_data , pos):
        self.save_data.append([input_data[0] , layer2_data[0] , layer3_data[0] , [output_data[0][pos]] , pos])  # since node lists in matrix form only take [0] element
        
    def save_neural_choices_reset(self):
        self.save_data = []
    
    def back_propogate(self , reinforcement):
        weight_list = [self.weight_input , self.weight_mid , self.weight_output]
        for moves in range(len(self.save_data)):
            for i in range(0,3):
                self.enforce(self.save_data[moves][i] , self.save_data[moves][i+1], weight_list[i],  self.save_data[moves][4] , reinforcement)
        
                
    def enforce(self , early_nodes , late_nodes, weight , pos , reinforcement):
        for early_node in range(len(early_nodes)):
            for late_node in range(len(late_nodes)):
                correct =  self.sigprime(early_nodes[early_node])  * (reinforcement - late_nodes[late_node]) * early_nodes[early_node]
                if len(late_nodes) == 1:
                    weight[early_node][pos] = weight[early_node][pos] + correct
                else:
                    weight[early_node][late_node] = weight[early_node][late_node] + correct
                
                
    def sigmoid(self , x):
        y = 1/(1 + np.exp(-x))
        return y
    
    def sigprime(self , x):
        s = self.sigmoid(x) 
        y = s * (1 - s)
        return y

In [436]:
# tries to come up with a probability from 0 to 1 for how likely player 1 will win
# only the final outcome is adjusted


class Predictor(Game):
    def __init__ (self , G = Game ,  weight_input = weights("input") , weight_mid = weights("mid") , weight_output = weights("pred_output") , save_data = []):
        self.weight_input = weight_input
        self.weight_mid = weight_mid
        self.weight_output = weight_output
        self.G = G
        self.save_data = save_data
        
    def prediction(self, reinforce = 0 , show = False):
        if self.G.over == True:
            self.back_propogate(reinforce)
        else:
            input_data = self.analyze(self.G.board)
            layer2_data = self.sigmoid(input_data.dot(self.weight_input))  # 1 X 9 times 9 X 12 --> 1 X 12
            layer3_data = self.sigmoid(layer2_data.dot(self.weight_mid))  # 1 X 12 times 12 X 12 --> 1 X 12
            output_data = self.sigmoid(layer3_data.dot(self.weight_output))  # 1 X 12 times 12 X 1 --> 1 X 1
            prediction = output_data[0][0]
            self.save_pred_choices(input_data , layer2_data , layer3_data , output_data)
            if show == True:
                print("prediction: " , prediction)
            return prediction
    
    def analyze(self , board):    
        input_data = np.array([list(map(float , list(board)))])   # makes the board as a string into number
        input_data = input_data.astype(float)
        np.place(input_data , input_data==2 , -1)   # opponent is viewed as low value
        np.place(input_data , input_data==0 , 0)   # empty space is regular value
        np.place(input_data , input_data==1 , 1)
        return input_data
    
    
    def save_pred_choices(self , input_data , layer2_data , layer3_data , output_data):
        self.save_data.append([input_data[0] , layer2_data[0] , layer3_data[0] , output_data[0]])  # since node lists in matrix form only take [0] element
        
    def save_pred_choices_reset(self):
        self.save_data = []
    
    def back_propogate(self , reinforcement):
        weight_list = [self.weight_input , self.weight_mid , self.weight_output]
        for moves in range(len(self.save_data)):
            for i in range(0,3):
                self.enforce(self.save_data[moves][i] , self.save_data[moves][i+1], weight_list[i], reinforcement)
        self.save_pred_choices_reset()
        
                
    def enforce(self , early_nodes , late_nodes, weight  , reinforcement):
        for early_node in range(len(early_nodes)):
            for late_node in range(len(late_nodes)):
                correct =  self.sigprime(early_nodes[early_node])  * (reinforcement - late_nodes[late_node]) * 0.05
                weight[early_node][late_node] = weight[early_node][late_node] + correct
                
    
    def sigmoid(self , x):
        y = 1/(1 + np.exp(-x))
        return y
    
    def sigprime(self , x):
        s = self.sigmoid(x) 
        y = s * (1 - s)
        return y
        
    

In [429]:
g = Game()
g.board

'000000000'

In [430]:
n1 = Neural(g)
n2 = Neural(g)
P = Predictor(g)

In [440]:
 g.play_again(5 , n2.neural_1 , g.rng , P.prediction, index = 1 , automate = True , show = True)

░░░
░░░
░░X
000000001
--------------------------------------------------

prediction:  0.66238514054
░O░
░░░
░░X
020000001
--------------------------------------------------

prediction:  0.649460468125
░OX
░░░
░░X
021000001
--------------------------------------------------

prediction:  0.662051014244
░OX
░░░
░OX
021000021
--------------------------------------------------

prediction:  0.64949476331
░OX
░░X
░OX
021001021
--------------------------------------------------

prediction:  0.662288958448
------------------
░░░
░░░
░░X
000000001
--------------------------------------------------

prediction:  0.997037774825
░O░
░░░
░░X
020000001
--------------------------------------------------

prediction:  0.995815857075
░OX
░░░
░░X
021000001
--------------------------------------------------

prediction:  0.997023740408
░OX
O░░
░░X
021200001
--------------------------------------------------

prediction:  0.993576774389
░OX
O░X
░░X
021201001
-------------------------------------------

array([[-0.39831868],
       [-0.07803976],
       [-0.50717416],
       [-0.9887026 ],
       [-0.50821155],
       [-0.69920392],
       [-0.35545811],
       [-0.68089928],
       [-0.32435704],
       [-0.88820394],
       [-0.17323705],
       [-0.62787851],
       [-0.43762325],
       [-0.12773204],
       [-0.32912851]])